In [1]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd

customer = pd.read_csv('sample_code/chapter_4/customer_join.csv')
uselog_months = pd.read_csv('dump/use_log_months.csv')

機械学習用の Data 加工は当月と１ヶ月前の Data の利用履歴のみの Data を作成する。
<small>
理由
- 過去６ヶ月分の Data から予測する場合、５ヶ月以内の退会は予測できない。
- ほんの数ヶ月で辞めてしまう顧客も多いので、過去６ヶ月分の Data からの予測では意味がない。

In [2]:
year_months = list(uselog_months['年月'].unique())  # 年月を List化
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]]
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i - 1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## 退会前月の Data を作成
退会当月の Data を用いても未然に防ぐことはできない為、前月の Data を用いて予測する。
### 理由・要因
1. 退会の予測をする目的
退会を未然に防ぐこと
2. Rule
当 Jim では、月末までに退会申請を提出することで、翌月末で退会できる。

In [3]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月'] = uselog['年月'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [4]:
# 欠損値を除外する
exit_uselog = exit_uselog.dropna(subset=['name'])
display(len(exit_uselog))
display(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104

1104

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


customer data は、end_date 以外に欠損値はないはず
    => customer data の名前列が欠損値であった場合 = 退会前月と結合できない不要な Data.

Data 件数と customer_id 列の unique 数が一致している為、顧客が辞める前月の状態を表している Data ということを確認。

# 継続顧客の Data を作成

In [6]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
display(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
display(len(conti_uselog))

33851

27422

Data | Data 数
--- | ---
退会 Data | 1104
継続顧客 Data | 27422

比較して退会 Data が少ない為、継続顧客の Data 全てを利用する場合、不均一な Data となってしまう。
=> Sample 数の調整が必要。

### 継続顧客も、顧客あたり１件になるよう Ander sampling を実施
Data を Shuffle して、重複を削除する方法をとる。

In [7]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True) # Data の Shuffle
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id') # customer_id が重複している Data は最初の Data のみ取得
display(len(conti_uselog))
conti_uselog.head()

2842

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201806,TS327886,4,7.0,XXXXX,C03,M,2017-04-01,NaN,CA1,...,ナイト,6000.0,通常,5.666667,6.0,8.0,3.0,1.0,2019-04-30,24.0
1,201808,HD052466,7,8.0,XX,C01,M,2017-12-01,NaN,CA3,...,オールタイム,10500.0,入会費無料,6.833333,6.5,9.0,5.0,1.0,2019-04-30,16.0
2,201903,AS495117,3,5.0,XXXXX,C01,M,2016-08-01,NaN,CA1,...,オールタイム,10500.0,通常,3.583333,3.0,6.0,1.0,1.0,2019-04-30,32.0
3,201806,PL551489,5,7.0,XXXX,C01,M,2018-02-01,NaN,CA1,...,オールタイム,10500.0,通常,6.666667,6.5,11.0,4.0,1.0,2019-04-30,14.0
4,201807,TS076452,5,4.0,XXXXX,C01,F,2016-10-01,NaN,CA1,...,オールタイム,10500.0,通常,4.750000,4.5,8.0,3.0,1.0,2019-04-30,30.0


継続顧客 Data が完成したので、退会顧客 Data と縦結合する

In [8]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201806,TS327886,4,7.0,XXXXX,C03,M,2017-04-01,NaN,CA1,...,6000.0,通常,5.666667,6.0,8.0,3.0,1.0,2019-04-30,24.0,NaT
1,201808,HD052466,7,8.0,XX,C01,M,2017-12-01,NaN,CA3,...,10500.0,入会費無料,6.833333,6.5,9.0,5.0,1.0,2019-04-30,16.0,NaT
2,201903,AS495117,3,5.0,XXXXX,C01,M,2016-08-01,NaN,CA1,...,10500.0,通常,3.583333,3.0,6.0,1.0,1.0,2019-04-30,32.0,NaT
3,201806,PL551489,5,7.0,XXXX,C01,M,2018-02-01,NaN,CA1,...,10500.0,通常,6.666667,6.5,11.0,4.0,1.0,2019-04-30,14.0,NaT
4,201807,TS076452,5,4.0,XXXXX,C01,F,2016-10-01,NaN,CA1,...,10500.0,通常,4.750000,4.5,8.0,3.0,1.0,2019-04-30,30.0,NaT
